In [1]:
"""
Surrogate-gradient Spiking Neural Network training script for MNIST / FMNIST

Features:
- Simple feedforward SNN with LIF neurons and surrogate gradient for the spike function
- Choice of dataset: mnist or fmnist
- Rate (scaled constant current) or Poisson encoding (selectable)
- Records: accuracy, F1 (macro), runtime per epoch and total
- Saves metrics to CSV and final model

Notes:
- Requires: torch, torchvision, tqdm, sklearn
- Run on a machine with internet to download datasets.

Example:
    python snn_surrogate_training_mnist_fmnist.py --dataset mnist --encoding rate --epochs 20 --batch-size 128

"""

import argparse
import time
import os
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST, FashionMNIST
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# ---------------------------
# Surrogate spike function
# ---------------------------
class SurrogateSpike(torch.autograd.Function):
    """Fast sigmoid surrogate gradient."""
    @staticmethod
    def forward(ctx, input, thresh=1.0):
        ctx.save_for_backward(input)
        ctx.thresh = thresh
        out = (input > thresh).float()
        return out

    @staticmethod
    def backward(ctx, grad_output):
        (input,) = ctx.saved_tensors
        thresh = ctx.thresh
        # derivative: a*(1 - |x-th|) approximation (triangular) or fast-sigmoid
        sigma = 1.0
        grad_input = grad_output.clone()
        # fast sigmoid derivative
        s = 1.0 / (1.0 + torch.exp(- (input - thresh) / sigma))
        grad = s * (1 - s) * (1.0 / sigma)
        return grad_input * grad, None

spike_fn = SurrogateSpike.apply

# ---------------------------
# LIF layer
# ---------------------------
class LIFLayer(nn.Module):
    def __init__(self, n_in, n_out, dt=1.0, tau_mem=20.0, thresh=1.0, decay=None):
        super().__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.thresh = thresh
        self.dt = dt
        self.tau_mem = tau_mem
        if decay is None:
            self.decay = torch.exp(torch.tensor(-dt / tau_mem)).item()
        else:
            self.decay = decay
        # Linear transform from input currents to membrane potential increment
        self.fc = nn.Linear(n_in, n_out, bias=False)

        # register buffer for decay so it moves with device
        self.register_buffer('decay_buf', torch.tensor(self.decay))

    def forward(self, x, mem, spike):
        # x: [batch, n_in] current input at this timestep
        # mem: [batch, n_out] previous membrane
        # spike: previous spikes (not used here but kept for API)
        cur = self.fc(x)
        mem = mem * self.decay_buf + cur
        out = spike_fn(mem, self.thresh)
        # reset membrane where spikes occurred (simple subtraction)
        mem = mem - out * self.thresh
        return mem, out

# ---------------------------
# Network
# ---------------------------
class SNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=400, n_classes=10,
                 time_steps=50, device='cpu'):
        super().__init__()
        self.device = device
        self.time_steps = time_steps
        self.hidden = LIFLayer(input_size, hidden_size)
        self.output = LIFLayer(hidden_size, n_classes)

    def forward(self, inputs):
        # inputs: [time_steps, batch, input_size]
        batch = inputs.size(1)
        device = inputs.device
        mem_h = torch.zeros(batch, self.hidden.n_out, device=device)
        mem_o = torch.zeros(batch, self.output.n_out, device=device)
        spike_h = torch.zeros(batch, self.hidden.n_out, device=device)
        spike_o = torch.zeros(batch, self.output.n_out, device=device)

        # accumulate output membrane to use as logits
        out_mem_accum = torch.zeros(batch, self.output.n_out, device=device)

        for t in range(self.time_steps):
            x_t = inputs[t]
            mem_h, spike_h = self.hidden(x_t, mem_h, spike_h)
            mem_o, spike_o = self.output(spike_h, mem_o, spike_o)
            out_mem_accum += mem_o

        # logits: accumulated membrane potentials over time
        return out_mem_accum

# ---------------------------
# Encoders
# ---------------------------
class RateEncoder:
    def __init__(self, time_steps, scale=5.0):
        self.time_steps = time_steps
        self.scale = scale

    def __call__(self, batch):
        # batch: [batch, 784] float in [0,1]
        cur = batch * self.scale
        # repeat across time
        return cur.unsqueeze(0).repeat(self.time_steps, 1, 1)

class PoissonEncoder:
    def __init__(self, time_steps):
        self.time_steps = time_steps

    def __call__(self, batch):
        # batch: [batch, 784] float in [0,1]
        bs = batch.size(0)
        return torch.rand(self.time_steps, bs, batch.size(1), device=batch.device) < batch.unsqueeze(0)

# ---------------------------
# Training / evaluation
# ---------------------------

def train_epoch(model, device, dataloader, optimizer, criterion, encoder):
    model.train()
    epoch_loss = 0.0
    all_preds = []
    all_targets = []
    t0 = time.time()

    for images, targets in tqdm(dataloader, desc='Train batches'):
        images = images.view(images.size(0), -1).to(device)
        images = (images / 255.0).float()
        targets = targets.to(device)

        inputs = encoder(images)
        if isinstance(inputs, torch.Tensor):
            inputs = inputs.to(device)
        else:
            inputs = inputs.float().to(device)

        optimizer.zero_grad()
        logits = model(inputs)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * images.size(0)

        preds = logits.argmax(dim=1).detach().cpu().numpy()
        all_preds.append(preds)
        all_targets.append(targets.cpu().numpy())

    epoch_time = time.time() - t0
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='macro')
    return epoch_loss / len(dataloader.dataset), acc, f1, epoch_time


def eval_epoch(model, device, dataloader, criterion, encoder):
    model.eval()
    epoch_loss = 0.0
    all_preds = []
    all_targets = []
    t0 = time.time()

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc='Eval batches'):
            images = images.view(images.size(0), -1).to(device)
            images = (images / 255.0).float()
            targets = targets.to(device)

            inputs = encoder(images)
            inputs = inputs.to(device)

            logits = model(inputs)
            loss = criterion(logits, targets)

            epoch_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1).cpu().numpy()
            all_preds.append(preds)
            all_targets.append(targets.cpu().numpy())

    epoch_time = time.time() - t0
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='macro')
    return epoch_loss / len(dataloader.dataset), acc, f1, epoch_time

# ---------------------------
# Utils / main
# ---------------------------

def get_dataloaders(dataset_name, batch_size):
    transform = transforms.ToTensor()
    if dataset_name == 'fmnist':
        train = MNIST(root='./data', train=True, download=True, transform=transform)
        test = MNIST(root='./data', train=False, download=True, transform=transform)
    else:
        train = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test = FashionMNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, test_loader


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', choices=['mnist', 'fmnist'], default='mnist')
    parser.add_argument('--encoding', choices=['rate', 'poisson'], default='rate')
    parser.add_argument('--hidden', type=int, default=400)
    parser.add_argument('--time-steps', type=int, default=10)
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--outdir', type=str, default='./snn_runs')
    parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
    args = parser.parse_args(args=[])


    device = torch.device(args.device)
    os.makedirs(args.outdir, exist_ok=True)

    train_loader, test_loader = get_dataloaders(args.dataset, args.batch_size)

    model = SNN(input_size=784, hidden_size=args.hidden, n_classes=10, time_steps=args.time_steps, device=device)
    model.to(device)

    if args.encoding == 'rate':
        encoder = RateEncoder(time_steps=args.time_steps, scale=5.0)
    else:
        encoder = PoissonEncoder(time_steps=args.time_steps)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss()

    metrics = []
    total_t0 = time.time()

    for epoch in range(1, args.epochs + 1):
        print(f"\nEpoch {epoch}/{args.epochs}")
        train_loss, train_acc, train_f1, train_time = train_epoch(model, device, train_loader, optimizer, criterion, encoder)
        test_loss, test_acc, test_f1, test_time = eval_epoch(model, device, test_loader, criterion, encoder)

        epoch_record = {
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'train_f1': train_f1,
            'train_time_s': train_time,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'test_f1': test_f1,
            'test_time_s': test_time,
        }
        metrics.append(epoch_record)

        print(f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f}, f1: {train_f1:.4f}, time: {train_time:.1f}s")
        print(f"Test  loss: {test_loss:.4f}, acc: {test_acc:.4f}, f1: {test_f1:.4f}, time: {test_time:.1f}s")

        # save intermediate model and metrics
        metrics_df = pd.DataFrame(metrics)
        metrics_df.to_csv(Path(args.outdir) / f"metrics_{args.dataset}_{args.encoding}.csv", index=False)
        torch.save(model.state_dict(), Path(args.outdir) / f"model_epoch{epoch}_{args.dataset}_{args.encoding}.pt")

    total_time = time.time() - total_t0
    print(f"\nTotal training+eval time: {total_time:.1f}s")

    # final save
    metrics_df.to_csv(Path(args.outdir) / f"metrics_final_{args.dataset}_{args.encoding}.csv", index=False)
    torch.save(model.state_dict(), Path(args.outdir) / f"model_final_{args.dataset}_{args.encoding}.pt")

    print(f"All done. Metrics and model saved to {args.outdir}")


if __name__ == '__main__':
    main()



Epoch 1/100


Eval batches: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 85.09it/s]


Train loss: 1.0225, acc: 0.6311, f1: 0.6343, time: 5.7s
Test  loss: 0.7364, acc: 0.7384, f1: 0.7345, time: 0.9s

Epoch 2/100


Eval batches: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 86.36it/s]


Train loss: 0.6869, acc: 0.7624, f1: 0.7595, time: 5.7s
Test  loss: 0.7008, acc: 0.7695, f1: 0.7638, time: 0.9s

Epoch 3/100


...

Train loss: 0.3776, acc: 0.8802, f1: 0.8802, time: 11.8s
Test  loss: 0.6698, acc: 0.8473, f1: 0.8470, time: 1.2s

Epoch 99/100


Eval batches: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 62.85it/s]


Train loss: 0.3744, acc: 0.8819, f1: 0.8819, time: 11.8s
Test  loss: 0.6798, acc: 0.8498, f1: 0.8489, time: 1.3s

Epoch 100/100


Eval batches: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 61.24it/s]

Train loss: 0.3738, acc: 0.8828, f1: 0.8828, time: 11.6s
Test  loss: 0.6798, acc: 0.8478, f1: 0.8467, time: 1.3s

Total training+eval time: 1535.0s
All done. Metrics and model saved to ./snn_runs
